Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
import cPickle as pickle
import numpy as np
import tensorflow as tf

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print 'Training set', train_dataset.shape, train_labels.shape
  print 'Validation set', valid_dataset.shape, valid_labels.shape
  print 'Test set', test_dataset.shape, test_labels.shape

Training set (400000, 28, 28) (400000,)
Validation set (30000, 28, 28) (30000,)
Test set (18724, 28, 28) (18724,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print 'Training set', train_dataset.shape, train_labels.shape
print 'Validation set', valid_dataset.shape, valid_labels.shape
print 'Test set', test_dataset.shape, test_labels.shape

Training set (400000, 784) (400000, 10)
Validation set (30000, 784) (30000, 10)
Test set (18724, 784) (18724, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compue the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [9]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # L2 regularization for the fully connected parameters.
  regularizers = (tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases))
  # Add the regularization term to the loss.
  loss += 4e-3 * regularizers  
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [10]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print "Initialized"
  for step in xrange(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print "Minibatch loss at step", step, ":", l
      print "Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels)
      print "Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels)
  print "Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels)

Initialized
Minibatch loss at step 0 : 29.2334
Minibatch accuracy: 11.7%
Validation accuracy: 15.0%
Minibatch loss at step 500 : 2.09416
Minibatch accuracy: 80.5%
Validation accuracy: 78.8%
Minibatch loss at step 1000 : 0.983963
Minibatch accuracy: 82.8%
Validation accuracy: 81.7%
Minibatch loss at step 1500 : 0.795776
Minibatch accuracy: 78.1%
Validation accuracy: 80.5%
Minibatch loss at step 2000 : 0.900181
Minibatch accuracy: 78.1%
Validation accuracy: 81.9%
Minibatch loss at step 2500 : 0.978477
Minibatch accuracy: 74.2%
Validation accuracy: 81.7%
Minibatch loss at step 3000 : 0.775216
Minibatch accuracy: 81.2%
Validation accuracy: 82.0%
Test accuracy: 88.4%


Now let's test regularization on a 1-hiddeny layer NN

In [11]:
batch_size = 128
hidden_layer_size = 1024

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_layer_size]))
    biases = tf.Variable(tf.zeros([hidden_layer_size]))
    
    # creating hidden layer
    hidden_layer_weights = tf.Variable(tf.truncated_normal([hidden_layer_size, num_labels]))
    hidden_layer_biases = tf.Variable(tf.zeros([num_labels]))
    
    def calculate_logits(dataset, weights1=weights, biases1=biases, 
                         weights2=hidden_layer_weights, biases2=hidden_layer_biases):
        hidden_layer_input = tf.nn.relu(tf.matmul(dataset, weights1) + biases1)
        logits = tf.matmul(hidden_layer_input, weights2) + biases2
        
        return logits
    
    # Training computation.
    logits = calculate_logits(tf_train_dataset)    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
   
    # L2 regularization for the fully connected parameters.
    regularizers = (tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases)
                   + tf.nn.l2_loss(hidden_layer_weights) + tf.nn.l2_loss(hidden_layer_biases))
    # Add the regularization term to the loss.
    loss += 4e-3 * regularizers
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_logits = calculate_logits(tf_valid_dataset)
    valid_prediction = tf.nn.softmax(valid_logits)
    test_logits = calculate_logits(tf_test_dataset)
    test_prediction = tf.nn.softmax(test_logits)

In [12]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print "Initialized"
  for step in xrange(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print "Minibatch loss at step", step, ":", l
      print "Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels)
      print "Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels)
  print "Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels)

Initialized
Minibatch loss at step 0 : 1590.29
Minibatch accuracy: 12.5%
Validation accuracy: 30.7%
Minibatch loss at step 500 : 167.996
Minibatch accuracy: 84.4%
Validation accuracy: 82.1%
Minibatch loss at step 1000 : 22.9476
Minibatch accuracy: 85.2%
Validation accuracy: 85.9%
Minibatch loss at step 1500 : 3.66676
Minibatch accuracy: 81.2%
Validation accuracy: 85.4%
Minibatch loss at step 2000 : 1.17008
Minibatch accuracy: 79.7%
Validation accuracy: 86.2%
Minibatch loss at step 2500 : 0.925597
Minibatch accuracy: 76.6%
Validation accuracy: 86.0%
Minibatch loss at step 3000 : 0.674001
Minibatch accuracy: 82.8%
Validation accuracy: 85.2%
Test accuracy: 91.4%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [85]:
batch_size = 600
hidden_layer_size = 1024

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_layer_size]))
    biases = tf.Variable(tf.zeros([hidden_layer_size]))
    
    # creating hidden layer
    hidden_layer_weights = tf.Variable(tf.truncated_normal([hidden_layer_size, num_labels]))
    hidden_layer_biases = tf.Variable(tf.zeros([num_labels]))
    
    def calculate_logits(dataset, weights1=weights, biases1=biases, 
                         weights2=hidden_layer_weights, biases2=hidden_layer_biases):
        hidden_layer_input = tf.nn.relu(tf.matmul(dataset, weights1) + biases1)
        logits = tf.matmul(hidden_layer_input, weights2) + biases2
        
        return logits
    
    # Training computation.
    logits = calculate_logits(tf_train_dataset)    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
   
    # L2 regularization for the fully connected parameters.
    regularizers = (tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases)
                   + tf.nn.l2_loss(hidden_layer_weights) + tf.nn.l2_loss(hidden_layer_biases))
    # Add the regularization term to the loss.
    loss += 4e-3 * regularizers
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_logits = calculate_logits(tf_valid_dataset)
    valid_prediction = tf.nn.softmax(valid_logits)
    test_logits = calculate_logits(tf_test_dataset)
    test_prediction = tf.nn.softmax(test_logits)

In [86]:
num_steps = 50

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print "Initialized"
  for step in xrange(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    offset = 0;
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print "Minibatch loss at step", step, ":", l
      print "Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels)
      print "Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels)
  print "Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels)

Initialized
Minibatch loss at step 0 : 1559.05
Minibatch accuracy: 11.5%
Validation accuracy: 40.6%
Test accuracy: 81.7%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [90]:
batch_size = 128
hidden_layer_size = 1024

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Initialize dropout
    keep_prob = tf.placeholder(tf.float32)
    
    # Variables.
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_layer_size]))
    biases = tf.Variable(tf.zeros([hidden_layer_size]))
    
    # creating hidden layer
    hidden_layer_weights = tf.Variable(tf.truncated_normal([hidden_layer_size, num_labels]))
    hidden_layer_biases = tf.Variable(tf.zeros([num_labels]))
    
    def calculate_logits(dataset, weights1=weights, biases1=biases, 
                         weights2=hidden_layer_weights, biases2=hidden_layer_biases, use_dropout=False):
        hidden_layer_input = tf.nn.relu(tf.matmul(dataset, weights1) + biases1)
        if use_dropout:
            hidden_layer_input = tf.nn.dropout(hidden_layer_input, keep_prob)
            
        logits = tf.matmul(hidden_layer_input, weights2) + biases2
        
        return logits
    
    # Training computation.
    logits = calculate_logits(tf_train_dataset)    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
   
    # L2 regularization for the fully connected parameters.
    regularizers = (tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases)
                   + tf.nn.l2_loss(hidden_layer_weights) + tf.nn.l2_loss(hidden_layer_biases))
    # Add the regularization term to the loss.
    loss += 4e-3 * regularizers
        
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_logits = calculate_logits(tf_valid_dataset)
    valid_prediction = tf.nn.softmax(valid_logits)
    test_logits = calculate_logits(tf_test_dataset)
    test_prediction = tf.nn.softmax(test_logits)

In [91]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print "Initialized"
  for step in xrange(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 0.7}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print "Minibatch loss at step", step, ":", l
      print "Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels)
      print "Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels)
  print "Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels)

Initialized
Minibatch loss at step 0 : 1529.41
Minibatch accuracy: 9.4%
Validation accuracy: 29.2%
Minibatch loss at step 500 : 168.002
Minibatch accuracy: 81.2%
Validation accuracy: 81.9%
Minibatch loss at step 1000 : 22.8997
Minibatch accuracy: 86.7%
Validation accuracy: 85.8%
Minibatch loss at step 1500 : 3.66437
Minibatch accuracy: 84.4%
Validation accuracy: 85.3%
Minibatch loss at step 2000 : 1.17232
Minibatch accuracy: 78.9%
Validation accuracy: 86.4%
Minibatch loss at step 2500 : 0.914922
Minibatch accuracy: 80.5%
Validation accuracy: 86.0%
Minibatch loss at step 3000 : 0.67725
Minibatch accuracy: 82.8%
Validation accuracy: 85.3%
Test accuracy: 91.5%


Now lets see how things change in the case of overfitting network

In [87]:
batch_size = 600
hidden_layer_size = 1024

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Initialize dropout
    keep_prob = tf.placeholder(tf.float32)
    
    # Variables.
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_layer_size]))
    biases = tf.Variable(tf.zeros([hidden_layer_size]))
    
    # creating hidden layer
    hidden_layer_weights = tf.Variable(tf.truncated_normal([hidden_layer_size, num_labels]))
    hidden_layer_biases = tf.Variable(tf.zeros([num_labels]))
    
    def calculate_logits(dataset, weights1=weights, biases1=biases, 
                         weights2=hidden_layer_weights, biases2=hidden_layer_biases, use_dropout=False):
        hidden_layer_input = tf.nn.relu(tf.matmul(dataset, weights1) + biases1)
        if use_dropout:
            hidden_layer_input = tf.nn.dropout(hidden_layer_input, keep_prob)
            
        logits = tf.matmul(hidden_layer_input, weights2) + biases2
        
        return logits
    
    # Training computation.
    logits = calculate_logits(tf_train_dataset)    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
   
    # L2 regularization for the fully connected parameters.
    regularizers = (tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases)
                   + tf.nn.l2_loss(hidden_layer_weights) + tf.nn.l2_loss(hidden_layer_biases))
    # Add the regularization term to the loss.
    loss += 4e-3 * regularizers
        
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_logits = calculate_logits(tf_valid_dataset)
    valid_prediction = tf.nn.softmax(valid_logits)
    test_logits = calculate_logits(tf_test_dataset)
    test_prediction = tf.nn.softmax(test_logits)

In [89]:
num_steps = 50

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print "Initialized"
  for step in xrange(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    offset = 0
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 0.7}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print "Minibatch loss at step", step, ":", l
      print "Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels)
      print "Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels)
  print "Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels)

Initialized
Minibatch loss at step 0 : 1568.72
Minibatch accuracy: 11.2%
Validation accuracy: 41.0%
Test accuracy: 83.0%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [96]:
batch_size = 128
hidden_layer_size = 1024
num_steps = 3001

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Initialize dropout
    keep_prob = tf.placeholder(tf.float32)
    
    # Variables.
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_layer_size]))
    biases = tf.Variable(tf.zeros([hidden_layer_size]))
    
    # creating hidden layer
    hidden_layer_weights = tf.Variable(tf.truncated_normal([hidden_layer_size, num_labels]))
    hidden_layer_biases = tf.Variable(tf.zeros([num_labels]))
    
    def calculate_logits(dataset, weights1=weights, biases1=biases, 
                         weights2=hidden_layer_weights, biases2=hidden_layer_biases, use_dropout=False):
        hidden_layer_input = tf.nn.relu(tf.matmul(dataset, weights1) + biases1)
        if use_dropout:
            hidden_layer_input = tf.nn.dropout(hidden_layer_input, keep_prob)
            
        logits = tf.matmul(hidden_layer_input, weights2) + biases2
        
        return logits
    
    # Training computation.
    logits = calculate_logits(tf_train_dataset)    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
   
    # L2 regularization for the fully connected parameters.
    regularizers = (tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases)
                   + tf.nn.l2_loss(hidden_layer_weights) + tf.nn.l2_loss(hidden_layer_biases))
    # Add the regularization term to the loss.
    loss += 4e-3 * regularizers
    
    # Setup learning rate with decay
    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, step, num_steps, 0.96)
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_logits = calculate_logits(tf_valid_dataset)
    valid_prediction = tf.nn.softmax(valid_logits)
    test_logits = calculate_logits(tf_test_dataset)
    test_prediction = tf.nn.softmax(test_logits)

In [97]:
with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print "Initialized"
  for step in xrange(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 0.7}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print "Minibatch loss at step", step, ":", l
      print "Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels)
      print "Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels)
  print "Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels)

Initialized
Minibatch loss at step 0 : 1629.19
Minibatch accuracy: 12.5%
Validation accuracy: 27.2%
Minibatch loss at step 500 : 182.266
Minibatch accuracy: 85.9%
Validation accuracy: 81.8%
Minibatch loss at step 1000 : 26.8274
Minibatch accuracy: 86.7%
Validation accuracy: 85.6%
Minibatch loss at step 1500 : 4.4578
Minibatch accuracy: 82.0%
Validation accuracy: 85.5%
Minibatch loss at step 2000 : 1.30485
Minibatch accuracy: 81.2%
Validation accuracy: 86.4%
Minibatch loss at step 2500 : 0.933592
Minibatch accuracy: 77.3%
Validation accuracy: 85.9%
Minibatch loss at step 3000 : 0.672622
Minibatch accuracy: 83.6%
Validation accuracy: 85.2%
Test accuracy: 91.4%


2-hidden layer NN + learning rate decay

In [257]:
batch_size = 64
hidden_layer1_size = 256
hidden_layer2_size = 32
hidden_layer3_size = 16
num_steps = 6001

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Initialize dropout
    keep_prob = tf.placeholder(tf.float32)
    
    # Variables.
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_layer1_size]))
    biases = tf.Variable(tf.zeros([hidden_layer1_size]))
    
    # creating hidden layer1
    hidden_layer1_weights = tf.Variable(tf.truncated_normal([hidden_layer1_size, hidden_layer2_size]))
    hidden_layer1_biases = tf.Variable(tf.zeros([hidden_layer2_size]))
    
    # creating hidden layer2
    hidden_layer2_weights = tf.Variable(tf.truncated_normal([hidden_layer2_size, hidden_layer3_size]))
    hidden_layer2_biases = tf.Variable(tf.zeros([hidden_layer3_size]))
    
    # creating hidden layer3
    hidden_layer3_weights = tf.Variable(tf.truncated_normal([hidden_layer3_size, num_labels]))
    hidden_layer3_biases = tf.Variable(tf.zeros([num_labels]))
    
    def calculate_logits(dataset, weights1=weights, biases1=biases, 
                         weights2=hidden_layer1_weights, biases2=hidden_layer1_biases,
                         weights3=hidden_layer2_weights, biases3=hidden_layer2_biases, 
                         weights4=hidden_layer3_weights, biases4=hidden_layer3_biases, use_dropout=False):
        relu1 = tf.nn.relu(tf.matmul(dataset, weights1) + biases1)
        if use_dropout:
            relu1 = tf.nn.dropout(relu1, keep_prob)
            
        relu2 = tf.nn.relu(tf.matmul(relu1, weights2) + biases2)
        #relu2 = tf.matmul(relu1, weights2) + biases2
        if use_dropout:
            relu2 = tf.nn.dropout(relu2, keep_prob)
        
        #relu3 = tf.nn.relu(tf.matmul(relu2, weights3) + biases3)
        relu3 = tf.matmul(relu2, weights3) + biases3
        
        if use_dropout:
            relu3 = tf.nn.dropout(relu3, keep_prob)
            
        logits = tf.matmul(relu3, weights4) + biases4
        
        return logits
    
    # Training computation.
    logits = calculate_logits(tf_train_dataset)    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
   
    # L2 regularization for the fully connected parameters.
    regularizers = (tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases)
                   + tf.nn.l2_loss(hidden_layer1_weights) + tf.nn.l2_loss(hidden_layer1_biases)
                   + tf.nn.l2_loss(hidden_layer2_weights) + tf.nn.l2_loss(hidden_layer2_biases)
                   + tf.nn.l2_loss(hidden_layer3_weights) + tf.nn.l2_loss(hidden_layer3_biases))
    # Add the regularization term to the loss.
    loss += 4e-4 * regularizers
    
    # Setup learning rate with decay
    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.003, step, num_steps, 0.96)
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_logits = calculate_logits(tf_valid_dataset)
    valid_prediction = tf.nn.softmax(valid_logits)
    test_logits = calculate_logits(tf_test_dataset)
    test_prediction = tf.nn.softmax(test_logits)

In [258]:
with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print "Initialized"
  for step in xrange(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 0.9}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print "Minibatch loss at step", step, ":", l
      print "Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels)
      print "Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels)
  print "Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels)

Initialized
Minibatch loss at step 0 : 2100.64
Minibatch accuracy: 6.2%
Validation accuracy: 16.2%
Minibatch loss at step 500 : 35.1783
Minibatch accuracy: 46.9%
Validation accuracy: 38.2%
Minibatch loss at step 1000 : 35.7597
Minibatch accuracy: 39.1%
Validation accuracy: 41.6%
Minibatch loss at step 1500 : 34.628
Minibatch accuracy: 40.6%
Validation accuracy: 38.7%
Minibatch loss at step 2000 : 35.1742
Minibatch accuracy: 23.4%
Validation accuracy: 37.1%
Minibatch loss at step 2500 : 34.454
Minibatch accuracy: 39.1%
Validation accuracy: 41.4%
Minibatch loss at step 3000 : 34.1978
Minibatch accuracy: 42.2%
Validation accuracy: 37.3%
Minibatch loss at step 3500 : 34.3217
Minibatch accuracy: 46.9%
Validation accuracy: 39.5%
Minibatch loss at step 4000 : 34.0851
Minibatch accuracy: 43.8%
Validation accuracy: 42.6%
Minibatch loss at step 4500 : 34.2535
Minibatch accuracy: 39.1%
Validation accuracy: 42.5%
Minibatch loss at step 5000 : 34.0751
Minibatch accuracy: 32.8%
Validation accuracy: 